In [27]:
!python -V

Python 3.13.2


Q1. Notebook
We'll start with the same notebook we ended up with in homework 1. We cleaned it a little bit and kept only the scoring part. You can find the initial notebook here.

Run this notebook for the March 2023 data.

What's the standard deviation of the predicted duration for this dataset?

1.24
6.24
12.28
18.28

A: the standard deviation of the predicted duration for this dataset is 6.247488852238703

In [9]:
import pickle
import pandas as pd
import numpy as np

In [29]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

c:\Users\Christian\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Christian\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [12]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [32]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [33]:
np.std(y_pred)#np.float64(6.247488852238703)

np.float64(6.247488852238703)

Q2. Preparing the output
What's the size of the output file?

36M
46M
56M
66M
Note: Make sure you use the snippet above for saving the file. It should contain only these two columns. For this question, don't change the dtypes of the columns and use pyarrow, not fastparquet.

In [34]:
year = 2023
month = 3
df_result = pd.DataFrame({
    'ride_id': f'{year:04d}/{month:02d}_' + df.index.astype('str'),
    'prediction': y_pred
})

In [35]:

df_result.to_parquet(
    './predictions.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [49]:
import pickle
import pandas as pd

categorical = ['PULocationID', 'DOLocationID']

def pipeline(model_path: str, year: int, month: int) -> float:

    with open(model_path, 'rb') as f_in:
        dv, model = pickle.load(f_in)

    df = pd.read_parquet(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = model.predict(X_val)

    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

    df_result = pd.DataFrame({
        'ride_id': df['ride_id'],
        'prediction': y_pred
    })

    output_file = f"predictions/result-{year}-{month}.parquet"
    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

    print(f"Saved predictions to {output_file}")
    return y_pred.mean()

if __name__ == "__main__":
    pipeline("model.bin", 2023, 3)



c:\Users\Christian\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Christian\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Saved predictions to predictions/result-2023-3.parquet


In [51]:
import os
print(os.path.getsize('./predictions/result-2023-3.parquet') / 1024**2, "MB")

61.05867099761963 MB


Q3. Creating the scoring script

Now let's turn the notebook into a script.

Which command you need to execute for that?

A: jupyter nbconvert --to python homework.ipynb

In [146]:
!jupyter nbconvert --to python homework.ipynb

[NbConvertApp] Converting notebook homework.ipynb to python
[NbConvertApp] Writing 2399 bytes to homework.py


In [1]:
!pip install --quiet pipenv

In [4]:
import sklearn
print(sklearn.__version__)

1.6.1


In [7]:
!python -m pipenv install mlflow scikit-learn==1.6.1

[    ] Creating virtual environment...
[=   ] Creating virtual environment...
[==  ] Creating virtual environment...
[=== ] Creating virtual environment...
[ ===] Creating virtual environment...
[  ==] Creating virtual environment...
[    ] Creating virtual environment...
[   =] Creating virtual environment...
[  ==] Creating virtual environment...
[ ===] Creating virtual environment...
[====] Creating virtual environment...

To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing mlflow...
Installation Succeeded
Installing scikit-learn==1.6.1...
Installation Succeeded
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (a2f100)...
All dependencies are now up-to-date!
Building requirements...
[    ] Locking packages...
Resolving dependencies...
[    ] Locking packages...
[=   ] Locking packages

Creating a virtualenv for this project
Pipfile: C:\Users\Christian\Documents\04-deployment\hw4\Pipfile
Using 
c:/Users/Christian/AppData/Local/Programs/Python/Python313/python.exe3.13.2 to 
create virtualenv...
created virtual environment CPython3.13.2.final.0-64 in 655ms
  creator CPython3Windows(dest=C:\Users\Christian\.virtualenvs\hw4-ns2QmUI_, 
clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, 
app_data_dir=C:\Users\Christian\AppData\Local\pypa\virtualenv)
    added seed packages: pip==25.1.1
  activators 
BashActivator,BatchActivator,FishActivator,NushellActivator,PowerShellActivator
,PythonActivator

Successfully created virtual environment!
Virtualenv location: C:\Users\Christian\.virtualenvs\hw4-ns2QmUI_
Creating a Pipfile for this project...
Pipfile.lock not found, creating...
Locking  dependencies...
Locking  dependencies...
Updated Pipfile.lock 
(494d5b4f482f0ef471f49afe28f00ec1a2ff75da2ce65060d8cabaeb3da2f100)!
Upgrad

Q4. Virtual environment
Now let's put everything into a virtual environment. We'll use pipenv for that.

Install all the required libraries. Pay attention to the Scikit-Learn version: it should be the same as in the starter notebook.

After installing the libraries, pipenv creates two files: Pipfile and Pipfile.lock. The Pipfile.lock file keeps the hashes of the dependencies we use for the virtual env.

What's the first hash for the Scikit-Learn dependency?

A: this hash: sha256:0650e730afb87402baa88afbf31c07b84c98272622aaba002559b614600ca691

Q5. Parametrize the script
Let's now make the script configurable via CLI. We'll create two parameters: year and month.

Run the script for April 2023.

What's the mean predicted duration?

7.29
14.29
21.29
28.29
Hint: just add a print statement to your script.

A: Mean predicted duration: 14.292282936862437

In [34]:
!python -m pipenv run python score.py --year 2023 --month 4

Reading data from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet...
Making predictions...
Saving results to predictions_yellow_2023-04.parquet...
63.14346981048584 MB
Mean predicted duration: 14.292282936862437


C:\Users\Christian\.virtualenvs\hw4-ns2QmUI_\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Christian\.virtualenvs\hw4-ns2QmUI_\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Q6. Docker container
Now run the script with docker. What's the mean predicted duration for May 2023?

0.19
7.24
14.24
21.19

In [35]:
!docker build -t test/score:latest .

#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 262B done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim
#2 DONE 1.0s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/5] FROM docker.io/agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim@sha256:f54535b73a8c3ef91967d5588de57d4e251b22addcbbfb6e71304a91c1c7027f
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 30B done
#5 DONE 0.0s

#6 [2/5] WORKDIR /app
#6 CACHED

#7 [3/5] COPY score.py .
#7 CACHED

#8 [4/5] RUN mkdir -p predictions
#8 CACHED

#9 [5/5] RUN pip install --no-cache-dir pandas pyarrow scikit-learn==1.5.0
#9 2.613 Collecting pandas
#9 2.754   Downloading pandas-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
#9 3.217      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 30.4 MB/s eta 0:00:00
#9 

In [36]:
!docker run --rm test/score:latest --year 2023 --month 5


Reading data from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet...
Making predictions...
Saving results to predictions_yellow_2023-05.parquet...
67.12875461578369 MB
Mean predicted duration: 0.19174419265916945
